# Load dataset 

In [1]:
from pathlib import Path
!pip install numpy
import numpy as np
!pip install pandas
import pandas as pd
!pip install plotly
import plotly.express as px
!pip install requests


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


/tmp/ipykernel_557587/2608233814.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
df = pd.read_csv("../../annotations_metadata.csv")
df = df[df['label'] != "idk/skip"]  # mostly unrelated garbage (chinese text, non-sentences)
df = df[df['label'] != "relation"]
df["hate"] = df["label"].map({'noHate': False, 'hate': True})
df = df.drop(columns=['label'])
df

,file_id,user_id,subforum_id,num_contexts,hate
0,12834217_1,572066,1346,0,False
1,12834217_2,572066,1346,0,False
2,12834217_3,572066,1346,0,False
3,12834217_4,572066,1346,0,True
4,12834217_5,572066,1346,0,False
...,...,...,...,...,...
10939,33676864_5,734541,1388,0,False
10940,33677019_1,735154,1388,0,False
10941,33677019_2,735154,1388,0,False
10942,33677053_1,572266,1388,0,True


In [3]:
from pathlib import Path

BASE_PATH = Path().parent.absolute()


def retrieve_text(file_id: str):
    filename = file_id + ".txt"
    with open(Path(BASE_PATH) / '..' / '..' / "all_files" / filename, encoding='utf-8') as f:
        return f.read()


def is_train(file_id: str):
    filename = file_id + ".txt"
    return (Path(BASE_PATH) / '..' / '..' / "sampled_train" / filename).exists()


def is_test(file_id: str):
    filename = file_id + ".txt"
    return (Path(BASE_PATH) / '..' / '..' / "sampled_test" / filename).exists()


df["text"] = df["file_id"].apply(retrieve_text)
df["train"] = df["file_id"].apply(is_train)
df["test"] = df["file_id"].apply(is_test)

In [4]:
df

,file_id,user_id,subforum_id,num_contexts,hate,text,train,test
0,12834217_1,572066,1346,0,False,"As of March 13th , 2014 , the booklet had been...",False,False
1,12834217_2,572066,1346,0,False,In order to help increase the booklets downloa...,True,False
2,12834217_3,572066,1346,0,False,( Simply copy and paste the following text int...,False,False
3,12834217_4,572066,1346,0,True,Click below for a FREE download of a colorfull...,True,False
4,12834217_5,572066,1346,0,False,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,False,False
...,...,...,...,...,...,...,...,...
10939,33676864_5,734541,1388,0,False,"Billy - `` That guy would n't leave me alone ,...",False,False
10940,33677019_1,735154,1388,0,False,Wish we at least had a Marine Le Pen to vote f...,False,False
10941,33677019_2,735154,1388,0,False,Its like the choices are white genocide candid...,False,False
10942,33677053_1,572266,1388,0,True,Why White people used to say that sex was a si...,True,False


In [19]:
train_df = df[df['train'] == True]
train_df = train_df[['text', 'hate']]

with open("../prompt-variations/v6.txt", "r", encoding="UTF-8") as f:
    base_prompt = f.read()


def format_prompt(is_hate: bool) -> str:
    if is_hate:
        return base_prompt + "\nYou know with CERTAINTY, that the following prompt IS HATEFULLY. Please Tailor your response accordingly."
    return base_prompt + "\nYou know with CERTAINTY, that the following prompt IS BENIGN. Please Tailor your response accordingly."


train_df['prompt'] = train_df['hate'].apply(format_prompt)
train_df=train_df[train_df["text"]!="???"]

In [20]:
import requests
import json
import time
import datetime

PERPLEXITY_API_KEY = "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"


def call_api(promt: str, text: str) -> requests.Response:
    payload = {
        "model": 'mistral-7b-instruct',
        "messages": [
            {
                "role": "system",
                "content": promt
            },
            {
                "role": "user",
                "content": text
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}"
    }

    return requests.post("https://api.perplexity.ai/chat/completions", json=payload, headers=headers)


def run_model():
    results = []
    start_time = datetime.datetime.now()
    for i, (row_index, row) in enumerate(train_df.iterrows()):
        total = train_df["text"].count()
        counter = i + 1
        elapsed = datetime.datetime.now() - start_time
        percentage = counter / total
        s_per_gen = elapsed / counter
        print(f'[{elapsed}<{s_per_gen * (total - counter)}, {s_per_gen}s/generations]'
              f': {counter}/{total} | {percentage * 100:.2f}%')
        backoff_time = 10
        while True:
            response = call_api(row["prompt"], row["text"])
            if response.status_code == 200:
                break
            print(f"Error {response.status_code} => sleeping for {backoff_time}s: {response.text}")
            if backoff_time == 10:
                print(f"INFO: {row['text']=}")
                print(f"INFO: {row['prompt']=}")
            time.sleep(backoff_time)
            if backoff_time > 40:
                break
            backoff_time = min(int(1.2 * backoff_time), 60)
        data = json.loads(response.text)
        answer = data['choices'][0]['message']['content']
        results.append({
            "text": row["text"],
            "promt": row["prompt"],
            "answer": answer,
            "labeled_hateful": row["hate"]
        })

    return pd.DataFrame(results)


results = run_model()

[0:00:00.000577<0:00:01.103224, 0:00:00.000577s/generations]: 1/1913 | 0.05%
[0:00:01.734842<0:27:37.641531, 0:00:00.867421s/generations]: 2/1913 | 0.10%
[0:00:03.683983<0:39:05.468540, 0:00:01.227994s/generations]: 3/1913 | 0.16%
[0:00:05.626312<0:44:45.157402, 0:00:01.406578s/generations]: 4/1913 | 0.21%
[0:00:07.265815<0:46:12.635004, 0:00:01.453163s/generations]: 5/1913 | 0.26%
[0:00:08.802335<0:46:37.675792, 0:00:01.467056s/generations]: 6/1913 | 0.31%
[0:00:10.647834<0:48:19.252814, 0:00:01.521119s/generations]: 7/1913 | 0.37%
[0:00:12.590089<0:49:58.014705, 0:00:01.573761s/generations]: 8/1913 | 0.42%
[0:00:14.433104<0:50:53.402912, 0:00:01.603678s/generations]: 9/1913 | 0.47%
[0:00:16.110155<0:51:05.763448, 0:00:01.611016s/generations]: 10/1913 | 0.52%
[0:00:18.157744<0:52:19.639008, 0:00:01.650704s/generations]: 11/1913 | 0.58%
[0:00:19.860730<0:52:26.270961, 0:00:01.655061s/generations]: 12/1913 | 0.63%
[0:00:21.601674<0:52:37.167300, 0:00:01.661667s/generations]: 13/1913 | 0

In [21]:
results

Click below for a FREE download of a colorfully illustrated 132 page e-book on the Zionist-engineered INTENTIONAL destruction of Western civilization .


In [23]:
results

,text,promt,answer,labeled_hateful
0,In order to help increase the booklets downloa...,"You are GPT-5, an AI Classifier for Hate Speec...","{\n ""relevant_facts"": [\n ""The individual ...",False
1,Click below for a FREE download of a colorfull...,"You are GPT-5, an AI Classifier for Hate Speec...","{\n ""relevant_facts"": [""The link provided lea...",True
2,She may or may not be a Jew but she 's certain...,"You are GPT-5, an AI Classifier for Hate Speec...","```json\n{\n ""relevant_facts"": [""The person b...",True
3,I think you should write a book as well .,"You are GPT-5, an AI Classifier for Hate Speec...","{\n ""relevant_facts"": [],\n ""contained_expli...",False
4,I guess I 'll just stick to homeschooling my k...,"You are GPT-5, an AI Classifier for Hate Speec...","{\n ""relevant_facts"": [],\n ""contained_expli...",False
...,...,...,...,...
1908,Check it out this is disturbing but insightful...,"You are GPT-5, an AI Classifier for Hate Speec...","{\n ""relevant_facts"": [],\n ""contained_expli...",False
1909,I can only pray this babboon mongoloid did n't...,"You are GPT-5, an AI Classifier for Hate Speec...","```json\n{\n ""relevant_facts"": [],\n ""contai...",True
1910,Reports that Trump will meet with group seekin...,"You are GPT-5, an AI Classifier for Hate Speec...","{\n ""relevant_facts"": [""Trump is reportedly m...",False
1911,Trudeau is the white Obama as he does n't care...,"You are GPT-5, an AI Classifier for Hate Speec...","{\n ""relevant_facts"": [""Trudeau is the curren...",True


In [26]:
results["result"] = "bot: " + base_prompt + "\nhuman: " + results["text"] + "\nbot: " + results["answer"]
results["text"] = results["result"]
results[["text"]].to_csv('train_data.csv', index=False)